In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import json
from time import time
import numpy as np
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, ngrams

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import DictVectorizer 
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SVMSMOTE, BorderlineSMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,mutual_info_classif
from sklearn.svm import SVC, libsvm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import numpy as np
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.linear_model import LogisticRegressionCV

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.libsvm module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from skle

In [ ]:
pooling_function = 'sum'
with open('/content/drive/My Drive/'+pooling_function+'-labels-intense-selection.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

word_freq = {}
with open('/content/drive/My Drive/unigram_freq.csv', 'r', encoding='utf-8') as file:
    count = 1
    for line in file:
      line  = line.strip().split(',')
      word_freq[line[0]] = count
      count += 1
      if count == 17000:
        break

In [ ]:
def tokenize(string):
  return string.split()

y = np.array([x[1] for x in data])
print(len(y))

start = time()
n1gram = 1
n2gram = 1
#v = DictVectorizer(sparse=True)
#v = CountVectorizer(ngram_range=(n1gram,n2gram))
v = TfidfVectorizer(ngram_range=(n1gram,n2gram), lowercase=False, sublinear_tf=True, tokenizer=tokenize, stop_words='english')
docs = [x[0] + ' '.join([j[1] for j in nltk.pos_tag(x[0].split())]) for x in data]
X_ = v.fit_transform(docs)
end = time()
tfidf_scores = v.vocabulary_end = time()

20000


In [ ]:
X_.shape

(20000, 55464)

Naive Bayes

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = MultinomialNB()

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 18.7019
classification time 0.0117
TPR Recall 0.7437
TNR specificity 0.6944
precision 0.7088
fmeasure 0.7258
accuracy 0.7191
bal acc 0.719
---------------------------------------------
0.7437 0.6944 0.7088 0.7258 0.7191 0.719
---------------------------------------------
tp 743.7
fp 305.6
fn 256.3
tn 694.4


Random Forest

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = RandomForestClassifier(n_estimators=300)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 18.7019
classification time 225.8008
TPR Recall 0.8296
TNR specificity 0.7057
precision 0.7383
fmeasure 0.7812
accuracy 0.7677
bal acc 0.7676
---------------------------------------------
0.8296 0.7057 0.7383 0.7812 0.7677 0.7676
---------------------------------------------
tp 829.6
fp 294.3
fn 170.4
tn 705.7


In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = SVC(max_iter=10000)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 18.7019


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/s

classification time 88.0926
TPR Recall 0.7855
TNR specificity 0.7422
precision 0.753
fmeasure 0.7688
accuracy 0.7638
bal acc 0.7638
---------------------------------------------
0.7855 0.7422 0.753 0.7688 0.7638 0.7638
---------------------------------------------
tp 785.5
fp 257.8
fn 214.5
tn 742.2


SGD

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = SGDClassifier(max_iter=2000, tol=1e-5)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 18.7019
classification time 0.6201
TPR Recall 0.7841
TNR specificity 0.7429
precision 0.7532
fmeasure 0.7682
accuracy 0.7635
bal acc 0.7635
---------------------------------------------
0.7841 0.7429 0.7532 0.7682 0.7635 0.7635
---------------------------------------------
tp 784.1
fp 257.1
fn 215.9
tn 742.9


KNN

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = KNeighborsClassifier(n_neighbors=200)

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 18.7019
classification time 2.0523
TPR Recall 0.7485
TNR specificity 0.6749
precision 0.6973
fmeasure 0.7219
accuracy 0.7117
bal acc 0.7117
---------------------------------------------
0.7485 0.6749 0.6973 0.7219 0.7117 0.7117
---------------------------------------------
tp 748.5
fp 325.1
fn 251.5
tn 674.9


Logistic Regression

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10
clf = LogisticRegressionCV()

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 18.7019


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


classification time 36.7461
TPR Recall 0.7696
TNR specificity 0.7522
precision 0.7565
fmeasure 0.7628
accuracy 0.7609
bal acc 0.7609
---------------------------------------------
0.7696 0.7522 0.7565 0.7628 0.7609 0.7609
---------------------------------------------
tp 769.6
fp 247.8
fn 230.4
tn 752.2


LSVC

In [ ]:
print('preprocessing time', round(end-start, 4))

kf = StratifiedKFold(n_splits = 10)
n_estimators = 10

clf = LinearSVC(random_state=0, tol=1e-10, loss='hinge')

accuracy = 0
balance = 0
recall = 0
precision = 0
specificity = 0
fmeasure = 0
TP = TN = FP = FN = 0
classification_time = 0
for train_index, test_index in kf.split(X_, y):
    X_train, X_test = X_[train_index], X_[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    startc = time()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)
    endc = time()
    classification_time += endc-startc
    #print('Confusion Matrix')
    #print(metrics.confusion_matrix(y_test, y_predicted))
    tn, fp, fn, tp = tuple(metrics.confusion_matrix(y_test, y_predicted).flatten())
    TP += tp
    TN += tn
    FP += fp
    FN += fn
    
    #print('Classification Reprot')
    #print(metrics.classification_report(y_test, y_predicted))

    #print('Accuracy')
    acc = (tp + tn) / (tp + tn + fp + fn)
    accuracy += acc
    
    pre = tp / (tp + fp)
    precision += pre
    
    rec = tp / (tp + fn)
    recall += rec
    
    spe = tn / (tn + fp)
    specificity += spe
    
    fm = (2 * pre * rec) / (pre + rec)
    fmeasure += fm

    bl = (rec + spe) / 2.
    balance += bl 

print('classification time', round(classification_time/10., 4))

print('TPR Recall', round(recall/10., 4))
print('TNR specificity', round(specificity/10.,4))
print('precision', round(precision/10.,4))
print('fmeasure', round(fmeasure/10.,4))
print('accuracy', round(accuracy/10.,4))
print('bal acc', round(balance/10.,4))
print('---------------------------------------------')
print(round(recall/10., 4), round(specificity/10.,4), round(precision/10.,4), round(fmeasure/10.,4), round(accuracy/10.,4), round(balance/10.,4))
print('---------------------------------------------')
print('tp', TP/10.)
print('fp', FP/10.)
print('fn', FN/10.)
print('tn', TN/10.)

preprocessing time 18.7019
classification time 0.3408
TPR Recall 0.7784
TNR specificity 0.741
precision 0.7504
fmeasure 0.764
accuracy 0.7597
bal acc 0.7597
---------------------------------------------
0.7784 0.741 0.7504 0.764 0.7597 0.7597
---------------------------------------------
tp 778.4
fp 259.0
fn 221.6
tn 741.0
